Download dataset from https://www.kaggle.com/datasets/andrewmvd/hard-hat-detection and unpack it into ./data/hard-hat-detection folder.


In [1]:
classes = ['helmet','head','person']
file_path = './data/hard-hat-detection/'
ann_path = file_path + 'annotations/'
img_path = file_path + 'images/'

I. Prepare data: copy to train and val folders, convert annotations from xml to txt

In [2]:
# segregate the dataset into train & validation (there are 5000 images)(train:4500, val:500)

from pathlib import Path

path_annotations = []
for i in Path(ann_path).glob('*.xml'):
    path_annotations.append(i)
path_annotations = sorted(path_annotations) #contains path to 5000 annots

path_images = []
for i in Path(img_path).glob('*.png'):
    path_images.append(i)
path_images = sorted(path_images) #contains path to 5000 images

In [3]:
# Split data : 90% Train, 10% Val
path_train_annot = path_annotations[:4500]
path_train_images = path_images[:4500]

path_val_annot = path_annotations[4500:5000]
path_val_images = path_images[4500:5000]

In [4]:
# making dirs to segregate train & val images & annotations & also to save best trained model
import os

# Creating directories to put train & val data
os.makedirs('./data/train/annotations',exist_ok = True)
os.makedirs('./data/train/images', exist_ok = True)
os.makedirs('./data/train/labels', exist_ok = True)
os.makedirs('./data/val/annotations', exist_ok = True)
os.makedirs('./data/val/images', exist_ok = True)
os.makedirs('./data/val/labels', exist_ok = True)

In [5]:
# copying images & xml annotations from input to working folder
import shutil
from tqdm import tqdm

for i, (path_annot, path_img) in tqdm(enumerate(zip(path_train_annot, path_train_images))):
    shutil.copy(path_img, './data/train/images/' + path_img.parts[-1])
    shutil.copy(path_annot, './data/train/annotations/' + path_annot.parts[-1])
    
for i, (path_annot, path_img) in tqdm(enumerate(zip(path_val_annot, path_val_images))):
    shutil.copy(path_img, './data/val/images/' + path_img.parts[-1])
    shutil.copy(path_annot, './data/val/annotations/' + path_annot.parts[-1])

4500it [00:14, 311.14it/s]
500it [00:01, 286.88it/s]


In [6]:
from xml.dom.minidom import parse
import numpy as np # linear algebra

def convert_annot(size , box):
    x1 = int(box[0])
    y1 = int(box[1])
    x2 = int(box[2])
    y2 = int(box[3])

    dw = np.float32(1. / int(size[0]))
    dh = np.float32(1. / int(size[1]))

    w = x2 - x1
    h = y2 - y1
    x = x1 + (w / 2)
    y = y1 + (h / 2)

    x = x * dw
    w = w * dw
    y = y * dh
    h = h * dh
    return [x, y, w, h]

def get_xml_data_and_save_as_txt(file_path, img_xml_file):
    img_path = file_path + '/annotations/' + img_xml_file + '.xml'
    #print(img_path)

    dom = parse(img_path)
    root = dom.documentElement
    img_name = root.getElementsByTagName("filename")[0].childNodes[0].data
    img_size = root.getElementsByTagName("size")[0]
    objects = root.getElementsByTagName("object")
    img_w = img_size.getElementsByTagName("width")[0].childNodes[0].data
    img_h = img_size.getElementsByTagName("height")[0].childNodes[0].data
    img_c = img_size.getElementsByTagName("depth")[0].childNodes[0].data
   
    img_box = []
    for box in objects:
        cls_name = box.getElementsByTagName("name")[0].childNodes[0].data
        x1 = int(box.getElementsByTagName("xmin")[0].childNodes[0].data)
        y1 = int(box.getElementsByTagName("ymin")[0].childNodes[0].data)
        x2 = int(box.getElementsByTagName("xmax")[0].childNodes[0].data)
        y2 = int(box.getElementsByTagName("ymax")[0].childNodes[0].data)
        
        img_box.append([cls_name, x1, y1, x2, y2])
  

   
    with open(file_path + '/labels/' + img_xml_file + '.txt' ,'a+') as file_path:
        for box in img_box:

            cls_num = classes.index(box[0])

            new_box = convert_annot( [img_w, img_h], box[1:])

            file_path.write(f"{cls_num} {new_box[0]} {new_box[1]} {new_box[2]} {new_box[3]}\n")

        file_path.flush()
        file_path.close()

def convert_annotations_to_labels(path):
    for file in os.listdir(path+'/annotations'): 
        get_xml_data_and_save_as_txt(path, file[:-4])

In [7]:
convert_annotations_to_labels('./data/val')
convert_annotations_to_labels('./data/train')

II. Setup Yolo 

In [ ]:
#Clone Yolo to ./yolov5 folder 
!git clone https://github.com/ultralytics/yolov5

In [14]:
##have no idea what's this
!wandb disabled

"wandb" �� ���� ����७��� ��� ���譥�
��������, �ᯮ��塞�� �ணࠬ��� ��� ������ 䠩���.


In [15]:
#start training with 416x416 image size, 30 epochs, data config from ./hard_head.yaml, model config from yolov5s.yaml and save weights to yolov5s.pt
!python ./yolov5/train.py --img 416 --batch 32 --epochs 30 --data ./hard_head.yaml --cfg ./yolov5/models/yolov5s.yaml --weights ./data/yolov5s.pt
#file yolov5s.pt should be copied to our Telegram bot server where we can run detection

III. Detection 

In [11]:
# detect images from ./data/val/images folder and use weight from yolov5s.pt and confidence 25%
!python ./yolov5/detect.py --source ./data/val/images  --weights ./data/yolov5s.pt --conf 0.25

python: can't open file 'c:\\Users\\y.zhukov\\shared\\Politech\\SafetyHelmetDetection\\detect.py': [Errno 2] No such file or directory
